In [2]:
using NCDatasets

## 1. Input parameters

In [18]:
Δlon = 1.
Δlat = 1.
longrid = -40.:Δlon:55.
latgrid = 24.:Δlat:67.
depthgrid = [0, 5, 10, 20, 30, 40, 50, 75, 100, 125, 150, 200, 250, 300, 400, 
    500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1750, 2000, 
    2500, 3000];
# Time should be generated from years and months
timegrid = "23787, 23876, 23968, 24060, 24152, 24241, 24333, 24425, 24517, 24606, 
    24698, 24790, 24882, 24972, 25064, 25156, 25248, 25337, 25429, 25521, 
    25613, 25702, 25794, 25886, 25978, 26067, 26159, 26251, 26343, 26433, 
    26525, 26617, 26709, 26798, 26890, 26982, 27074, 27163, 27255, 27347, 
    27439, 27528, 27620, 27712, 27804, 27894, 27986, 28078, 28170, 28259, 
    28351, 28443, 28535, 28624, 28716, 28808, 28900, 28989, 29081, 29173, 
    29265, 29355, 29447, 29539, 29631, 29720, 29812, 29904, 29996, 30085, 
    30177, 30269, 30361, 30450, 30542, 30634, 30726, 30816, 30908, 31000, 
    31092, 31181, 31273, 31365, 31457, 31546, 31638, 31730, 31822, 31911, 
    32003, 32095, 32187, 32277, 32369, 32461, 32553, 32642, 32734, 32826, 
    32918, 33007, 33099, 33191, 33283, 33372, 33464, 33556, 33648, 33738, 
    33830, 33922, 34014, 34103, 34195, 34287, 34379, 34468, 34560, 34652, 
    34744, 34833, 34925, 35017, 35109, 35199, 35291, 35383, 35475, 35564, 
    35656, 35748, 35840, 35929, 36021, 36113, 36205, 36294, 36386, 36478, 
    36570, 36660, 36752, 36844, 36936, 37025, 37117, 37209, 37301, 37390, 
    37482, 37574, 37666, 37755, 37847, 37939, 38031, 38121, 38213, 38305, 
    38397, 38486, 38578, 38670, 38762, 38851, 38943, 39035, 39127, 39216, 
    39308, 39400, 39492, 39582, 39674, 39766, 39858, 39947, 40039, 40131, 
    40223, 40312, 40404, 40496";

In [11]:
varname = "chlorophyll-a"
product_id = "e61d12cd-837f-49ff-a0e1-3a694ab84bc5"
outputdir = "/data/EMODnet/Chemistry/combined"

"/data/EMODnet/Chemistry/combined"

Create directory, generate output file name and title.

In [19]:
if !(isdir(outputdir))
    @info("Create new output directory")
    mkpath(outputdir)
else
    @info("Output directory already exists")
end
outputfile = "Water_body_$(varname)_combined.nc"
@info(outputfile)
outputtitle = "DIVA 4D analysis of Water_body_$(varname)"
@info(outputtitle)

┌ Info: Output directory already exists
└ @ Main In[19]:5
┌ Info: Water_body_chlorophyll-a_combined.nc
└ @ Main In[19]:8
┌ Info: DIVA 4D analysis of Water_body_chlorophyll-a
└ @ Main In[19]:10


## 2. Create merged netCDF (empty)
The code below is generated (and adapted) from the output of
```julia
ncgen(templatefile)
```

In [24]:
ds = Dataset(joinpath(outputdir, outputfile), "c")

# Dimensions
ds.dim["lon"] = length(longrid)
ds.dim["nv"] = 2
ds.dim["lat"] = length(latgrid)
ds.dim["depth"] = length(depthgrid)
ds.dim["time"] = length(timegrid)

# Declare variables
nclon = defVar(ds,"lon", Float64, ("lon",))
nclon.attrib["standard_name"] = "longitude"
nclon.attrib["long_name"] = "longitude"
nclon.attrib["units"] = "degrees east"
nclon.attrib["axis"] = "X"
nclon.attrib["valid_min"] = -180.
nclon.attrib["valid_max"] = 180.

nclon_bnds = defVar(ds,"lon_bnds", Float64, ("nv", "lon"))

nclat = defVar(ds,"lat", Float64, ("lat",))
nclat.attrib["standard_name"] = "latitude"
nclat.attrib["long_name"] = "latitude"
nclat.attrib["units"] = "degrees north"
nclat.attrib["axis"] = "Y"
nclat.attrib["valid_min"] = -90.
nclat.attrib["valid_max"] = 90.

nclat_bnds = defVar(ds,"lat_bnds", Float64, ("nv", "lat"))

ncdepth = defVar(ds,"depth", Float64, ("depth",))
ncdepth.attrib["standard_name"] = "depth"
ncdepth.attrib["units"] = "meters"
ncdepth.attrib["axis"] = "Z"
ncdepth.attrib["positive"] = "down"

ncdepthnew_bnds = defVar(ds,"depthnew_bnds", Float64, ("nv", "depth"))

nctime = defVar(ds,"time", Float64, ("time",))
nctime.attrib["standard_name"] = "time"
nctime.attrib["bounds"] = "time3_bnds"
nctime.attrib["units"] = "days since 1900-01-01 00:00:00"
nctime.attrib["calendar"] = "standard"
nctime.attrib["climatology"] = "climatology_bounds"

ncclimatology_bounds = defVar(ds,"climatology_bounds", Float64, ("nv", "time"))
ncclimatology_bounds.attrib["units"] = "days since 1900-01-01 00:00:00"
ncclimatology_bounds.attrib["calendar"] = "standard"

# Interpolated variable
# (should be obtained from one of the regional netCDF files)
ncvarinterp = defVar(ds,"Water_body_ammonium", Float32, ("lon", "lat", "depth", "time"))
ncvarinterp.attrib["long_name"] = "Water_body_ammonium"
ncvarinterp.attrib["_FillValue"] = Float32(-9999.)
ncvarinterp.attrib["missing_value"] = Float32(-9999.)
ncvarinterp.attrib["actual_range"] = "0,125"
ncvarinterp.attrib["units"] = "umol/l"


# Global attributes
ds.attrib["Conventions"] = "CF-1.0"
ds.attrib["title"] = "DIVA 4D analysis of Water_body_ammonium"
ds.attrib["product_id"] = product_id
ds.attrib["abstract"] = "Merge netCDF product obtained from the Arctic, 
    Atlantic, Baltic, Black, Mediterranean and North seas"
ds.attrib["Creation date"] = 
close(ds)

In [26]:
templatefile = "./Water_body_ammonium.combined.nc"

"./Water_body_ammonium.combined.nc"

In [27]:
ncgen(templatefile)

ds = Dataset("filename.nc","c")
# Dimensions

ds.dim["lon"] = 950
ds.dim["nv"] = 2
ds.dim["lat"] = 430
ds.dim["depth"] = 30
ds.dim["time"] = 184

# Declare variables

nclon = defVar(ds,"lon", Float64, ("lon",))
nclon.attrib["standard_name"] = "longitude"
nclon.attrib["long_name"] = "longitude"
nclon.attrib["units"] = "degrees east"
nclon.attrib["axis"] = "X"
nclon.attrib["bounds"] = "lon_bnds"

nclon_bnds = defVar(ds,"lon_bnds", Float64, ("nv", "lon"))

nclat = defVar(ds,"lat", Float64, ("lat",))
nclat.attrib["standard_name"] = "latitude"
nclat.attrib["long_name"] = "latitude"
nclat.attrib["units"] = "degrees north"
nclat.attrib["axis"] = "Y"
nclat.attrib["bounds"] = "lat_bnds"

nclat_bnds = defVar(ds,"lat_bnds", Float64, ("nv", "lat"))

ncdepth = defVar(ds,"depth", Float64, ("depth",))
ncdepth.attrib["units"] = "meters"
ncdepth.attrib["axis"] = "Z"
ncdepth.attrib["bounds"] = "depthnew_bnds"
ncdepth.attrib["positive"] = "down"

ncdepthnew_bnds = defVar(ds,"depthnew_bnds", Float64, ("nv